In [2]:
import os
import cv2
from scipy.spatial import procrustes

import numpy as np
import pandas as pd

target_dir = "../data/single_sungaya/"

out_df = pd.read_hdf("../data/single_sungaya/Data_3D_Pose.hdf5")
out_df

,file_name,cam_rot,cam_trans,cam_intrinsics,bounding_box,key_points_3D,key_points_2D,visibility
0,1002_Img_synth.png,"[[0.299712, -0.685369, -0.663658], [-0.95103, ...","[-206.887, -85.8906, 871.54]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[0, 84, 414, 595]","[[-3.047812, -269.825287, 96.387993], [-10.202...","[[95.407867, 384.01593], [129.091675, 351.2847...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1005_Img_synth.png,"[[0.432294, 0.464133, 0.773112], [0.707314, -0...","[50.4178, -487.672, 600.386]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[223, 156, 640, 640]","[[231.146606, -368.931488, 96.387993], [246.39...","[[502.015533, 526.509338], [489.601746, 452.79...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1006_Img_synth.png,"[[-0.0613171, 0.973666, 0.21958], [0.817173, 0...","[-366.074, -101.078, 698.515]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[185, 231, 640, 589]","[[49.874466, 377.015411, 96.387993], [41.18921...","[[367.270233, 436.59491], [412.659454, 438.565...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1011_Img_synth.png,"[[0.980659, 0.137138, -0.13965], [0.145579, -0...","[208.431, -47.2549, 640.152]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[0, 96, 384, 615]","[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...","[[0, 0], [0, 0], [0, 0], [0, 0], [204.477234, ...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1015_Img_synth.png,"[[-0.968932, -0.23072, -0.0891025], [-0.238785...","[-101.129, -125.773, 646.068]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[145, 0, 607, 401]","[[-184.588226, 150.16156, 96.387993], [-167.59...","[[402.672729, 101.495071], [371.821045, 139.67...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...
3773,9981_Img_synth.png,"[[0.703921, -0.425377, 0.568814], [-0.710277, ...","[43.5846, 91.6788, 864.243]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[206, 92, 640, 429]","[[37.699928, 197.113602, 96.387993], [47.36378...","[[421.396729, 229.956238], [449.737854, 234.33...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3774,9994_Img_synth.png,"[[-0.775938, 0.485034, -0.403313], [0.428827, ...","[-39.4346, 121.901, 591.735]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[0, 0, 480, 554]","[[-116.847282, -72.601166, 96.387993], [-95.30...","[[239.093399, 213.272491], [186.209488, 192.51...","[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3775,9995_Img_synth.png,"[[-0.777703, -0.54052, 0.320961], [-0.58631, 0...","[-136.092, -193.223, 454.104]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[138, 116, 640, 640]","[[0, 0, 0], [0, 0, 0], [-275.554962, 109.75146...","[[0, 0], [0, 0], [631.721497, 620.308716], [0,...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3776,9999_Img_synth.png,"[[0.984181, -0.131343, -0.118895], [-0.0615904...","[-257.347, -729.683, 270.749]","[[1600.000244, 0.0, 320.0], [0.0, 1600.000244,...","[0, 0, 498, 479]","[[105.463318, -854.950378, 96.387993], [125.97...","[[167.771988, 136.572433], [224.189758, 161.59...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [3]:
out_df = pd.read_hdf("../data/single_sungaya/Data_3D_Pose.hdf5")
array_3d = np.array(out_df['key_points_3D'].to_numpy())

for x in range(len(array_3d)):
    array_3d[x] = np.array(array_3d[x])

fixed_array_3d = np.empty((3778,186))

for x in range(len(fixed_array_3d)):
    z = array_3d[x].reshape(1,186)
    fixed_array_3d[x] = z

fixed_array_3d=fixed_array_3d.reshape((3778,62,3))

In [6]:
avg= 0
disp=[]
for i in range(len(fixed_array_3d)):
    for j in range(i+1, len(fixed_array_3d)):
        if fixed_array_3d[i].all()==0 or fixed_array_3d[j].all()==0:
            x=0
        else:    
            _,_,dis =procrustes(fixed_array_3d[i], fixed_array_3d[j])
            avg += dis
            disp.append(dis)
print("Spacial disparity Avg:", avg/(len(fixed_array_3d)**2))
print(len(disp))

Spacial disparity Avg: 0.005259355052387793
682696
